In [5]:
import numpy as np
import scipy.sparse.linalg as splinalg
from scipy import interpolate
import matplotlib.pyplot as plt
import cmasher as cmr
from tqdm import tqdm
import math

In [9]:

#GRID
SPACES = 0.01
CELLS_X = 200
CELLS_Y = 100
POINTS_X = CELLS_X+1
POINTS_Y = CELLS_Y+1

#Domain
DOMAIN_X_SIZE = CELLS_X * SPACES
DOMAIN_Y_SIZE = CELLS_Y * SPACES

#Obstacle
IS_OBSTACLE = True
IS_OBSTACLE_MOVE = False
OBSTACLE_TYPE = 'cylinder'
OBSTACLE_CENTER_X = 0.3 * DOMAIN_X_SIZE
OBSTACLE_CENTER_Y = 0.5 * DOMAIN_Y_SIZE
OBSTACLE_R = 0.2 * min(DOMAIN_Y_SIZE,DOMAIN_X_SIZE)

#Solve
TIME_STEP_LENGTH = 0.001


In [11]:
class Grid:
    def __init__(self,DOMAIN_X_SIZE,DOMAIN_Y_SIZE,SPACES,POINTS_X,POINTS_Y):
        self.lx = DOMAIN_X_SIZE
        self.ly = DOMAIN_Y_SIZE
        self.dh = SPACES
        self.nx = POINTS_X
        self.ny = POINTS_Y
        self.x = np.linspace(0, self.lx, self.nx)
        self.y = np.linspace(0, self.ly, self.ny)

In [12]:
class DATA:
    def __init__(self,x_start,x_end,nx,y_start,y_end,ny):
        self.crood_x = np.linspace(x_start,x_end,nx)
        self.crood_y = np.linspace(y_start,y_end,nx)
        self.data = np.ones((nx, ny))
        

In [14]:
class Field:
    def __init__(self,DATA_NAME,DATA_TYPE,IS_GOST_CELLS,POINTS_X,POINTS_Y):
        self.name = DATA_NAME
        self.type = DATA_TYPE
        if self.type == 'scalar':
            if self.IS_GOST_CELLS == False:
                self.nx = POINTS_X-1
                self.ny = POINTS_Y-1
                self.scalar = DATA(0.5*SPACES,DOMAIN_X_SIZE + 0.5*SPACES,self.nx,0.5*SPACES,DOMAIN_Y_SIZE + 0.5*SPACES,self.ny)
        self.IS_GOST_CELLS = IS_GOST_CELLS
        
        